# **Logistic Code**

In [ ]:
#@title Imports

import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import glob # The glob module is used for Unix style pathname pattern expansion.
import imageio # The library that provides an easy interface to read and write a wide range of image data
import os
import PIL
import time
import gdown

from IPython import display

In [ ]:
#@title Load MNIST Database

# Using underscore to omit the label arrays
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data() 


train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
#@title Generator Model

def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    print(model.summary())

    return model

generator = generator_model()

In [ ]:
# Create a random noise and generate a sample
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
# Visualize the generated sample
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
#@title Discriminator Model

def discriminator_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    print(model.summary())

    return model

discriminator = discriminator_model()

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# **Training Code**

In [ ]:
# @title Training Steps

# EPOCHS = 3000 #@param{type: "integer"} 
# num_examples_to_generate = 16
# noise_dim = 100
# seed = tf.random.normal([num_examples_to_generate, noise_dim])

# # tf.function annotation causes the function to be "compiled" as part of the training
# @tf.function
# def train_step(images):
  
#     # 1 - Create a random noise to feed it into the model for the image generation
#     noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
#     # 2 - Generate images and calculate loss values
#     # The method records operations for automatic differentiation
#     with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
#       generated_images = generator(noise, training=True)

#       real_output = discriminator(images, training=True)
#       fake_output = discriminator(generated_images, training=True)

#       gen_loss = generator_loss(fake_output)
#       disc_loss = discriminator_loss(real_output, fake_output)

#     # 3 - Calculate gradients using loss values and model variables
#     # "gradient" method computes the gradient using operations recorded in context of this tape (gen_tape and disc_tape).
    
#     # It accepts a target (e.g., gen_loss) variable and a source variable (e.g.,generator.trainable_variables)
#     # target --> a list or nested structure of Tensors or Variables to be differentiated.
#     # source --> a list or nested structure of Tensors or Variables.
#     # target will be differentiated against elements in sources.

#     # "gradient" method returns a list or nested structure of Tensors or IndexedSlices, or None, one for each element in sources. 
#     # Returned structure is the same as the structure of sources.
#     gradients_of_generator = gen_tape.gradient(gen_loss, 
#                                                generator.trainable_variables)
#     gradients_of_discriminator = disc_tape.gradient(disc_loss, 
#                                                 discriminator.trainable_variables)
    
#     # 4 - Process  Gradients and Run the Optimizer
#     # "apply_gradients" method processes aggregated gradients. 
#     """
#     Example to show use of apply_gradients:
#     grads = tape.gradient(loss, vars)
#     grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
#     # Processing aggregated gradients.
#     optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
#     """
#     generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
#     discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
# @title Function to Generate Images

# def generate_and_save_images(model, epoch, test_input):
#   # 1 - Generate images
#   predictions = model(test_input, training=False)

#   # 2 - Plot the generated images
#   fig = plt.figure(figsize=(4,4))
#   for i in range(predictions.shape[0]):
#       plt.subplot(4, 4, i+1)
#       plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
#       plt.axis('off')

#   # 3 - Save the generated images
#   plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))

#   # 4 - Display the generated images
#   plt.show()

In [ ]:
# @title Train GAN

# import time
# from IPython import display # A command shell for interactive computing in Python.

# def train(dataset, epochs):
#   # A. For each epoch, do the following:
#   for epoch in range(epochs):
#     start = time.time()
#     # 1 - For each batch of the epoch, 
#     for image_batch in dataset:
#       # Run the custom "train_step" function we just declared above
#       train_step(image_batch)

#     # 2 - Produce images for the GIF as we go
#     display.clear_output(wait=True)
#     generate_and_save_images(generator,
#                              epoch + 1,
#                              seed)

#     # 3 - Print out the completed epoch no. and the time spent
#     print ('Epoch {} took {} seconds'.format(epoch + 1, time.time()-start))

#   # B. Generate a final image after the training is completed
#   display.clear_output(wait=True)
#   generate_and_save_images(generator,
#                            epochs,
#                            seed)
  
# train(train_dataset, EPOCHS)

In [ ]:
# @title Save model to GDrive

# from google.colab import drive
# drive.mount('/content/gdrive')

# gen_save_name = 'generator' 
# gen_save_name += '.tf'  
# disc_save_name = 'discriminator' 
# disc_save_name += '.tf' 

# path = 'My Drive/Computational_Creativity/models/' 
# full_path_g = F"/content/gdrive/{path}{gen_save_name}" 
# full_path_d = F"/content/gdrive/{path}{disc_save_name}" 

# generator.save(full_path_g, save_format='tf')
# discriminator.save(full_path_d, save_format='tf')

# **Generation Code**

In [ ]:
#@title Download Saved Model

url = "https://drive.google.com/drive/folders/1A5DKYRanMbaRs-oH35yOjBbBd6mqPPqK"
download_successful = None 
while download_successful == None:
  download_successful = gdown.download_folder(url, quiet=True, use_cookies=False)
  os.system('rm ~/.cache/gdown/cookies.json')
full_path_g = "/content/models/generator.tf"
full_path_d = "/content/models/discriminator.tf"
generator = tf.keras.models.load_model(full_path_g)
discriminator = tf.keras.models.load_model(full_path_d)

In [ ]:
#@title Show Images

SIZE=16
for samples in range(0,10):
  print("Sample:", samples)
  noise = tf.random.normal([SIZE, 100])
  predictions = generator.predict(noise)
  fig = plt.figure(figsize=(4,4))
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
    # Locally save images for the runtime
  plt.savefig('image_at_epoch_{:04d}.png'.format(samples))
  plt.show()

In [ ]:
#@title Create GIF

# import glob # The glob module is used for Unix style pathname pattern expansion.
# import imageio # The library that provides an easy interface to read and write a wide range of image data

anim_file = 'handwritten.gif'

with imageio.get_writer(anim_file, mode='I') as writer2:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    # print(filename)
    image = imageio.imread(filename)
    writer2.append_data(image)
  # image = imageio.imread(filename)
  # writer.append_data(image)
  
display.Image(open('handwritten.gif','rb').read())